In [5]:
#from tracker import Tracker
import copy
import colorsys
import os,sys,argparse,random
from timeit import default_timer as timer
import cv2
import numpy as np
from keras import backend as K
from keras.models import load_model
from keras.layers import Input
from PIL import Image, ImageFont, ImageDraw

from yolo3.model import yolo_eval, yolo_body, tiny_yolo_body
from yolo3.utils import letterbox_image
from keras.utils import multi_gpu_model
from yolo_matt import YOLO, detect_video

from tqdm import tqdm
from scipy import misc

%matplotlib inline 
from matplotlib import pyplot as plt

from objecttracker.KalmanFilterTracker import Tracker  # 加载卡尔曼滤波函数

def calc_center(out_boxes,out_classes,out_scores,score_limit = 0.5):
    outboxes_filter = []
    for x,y,z in zip(out_boxes,out_classes,out_scores):
        if z > score_limit:
            if y == 5 or y == 2 or y == 7: #  bus 5, car 2, truck, 7
                outboxes_filter.append(x)
    
    centers= []
    number = len(outboxes_filter)
    for box in outboxes_filter:
        top, left, bottom, right = box
        center=np.array([[(left+right)//2],[(top+bottom)//2]])
        centers.append(center)
    return centers,number


def get_colors_for_classes(num_classes):
    """Return list of random colors for number of classes given."""
    # Use previously generated colors if num_classes is the same.
    if (hasattr(get_colors_for_classes, "colors") and
            len(get_colors_for_classes.colors) == num_classes):
        return get_colors_for_classes.colors

    hsv_tuples = [(x / num_classes, 1., 1.) for x in range(num_classes)]
    colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
    colors = list(
        map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),
            colors))
    #colors = [(255,99,71) if c==(255,0,0) else c for c in colors ]  # 单独修正颜色，可去除
    random.seed(10101)  # Fixed seed for consistent colors across runs.
    random.shuffle(colors)  # Shuffle colors to decorrelate adjacent classes.
    random.seed(None)  # Reset seed to default.
    get_colors_for_classes.colors = colors  # Save colors for future calls.
    return colors

def trackerDetection(tracker,image,centers,number,max_point_distance = 30,max_colors = 30,track_id_size = 0.3):
    '''
        - max_point_distance为两个点之间的欧式距离不能超过30
            - 有多条轨迹,tracker.tracks;
            - 每条轨迹有多个点,tracker.tracks[i].trace
        - max_colors,最大颜色数量
        - track_id_size,每个中心点，显示的标记字体大小
    '''
    #track_colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0),
    #            (0, 255, 255), (255, 0, 255), (255, 127, 255),
    #            (127, 0, 255), (127, 0, 127)]
    track_colors = get_colors_for_classes(max_colors)
    
    result = np.asarray(image)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(result, str(number), (20,  40), font, 1, (0, 0, 255), 5)  # 左上角，人数计数

    if (len(centers) > 0):
        # Track object using Kalman Filter
        tracker.Update(centers)#dictionary
        # For identified object tracks draw tracking line
        # Use various colors to indicate different track_id
        for i in range(len(tracker.tracks)):
            # 多个轨迹
            if (len(tracker.tracks[i].trace) > 1):
                x0,y0 = tracker.tracks[i].trace[-1][0][0],tracker.tracks[i].trace[-1][1][0]
                cv2.putText(result,str(tracker.tracks[i].track_id),(int(x0),int(y0)),font,track_id_size,(255, 255, 255),2)  
                # (image,text,(x,y),font,size,color,粗细)
                for j in range(len(tracker.tracks[i].trace) - 1):
                    #每条轨迹的每个点
                    # Draw trace line
                    x1 = tracker.tracks[i].trace[j][0][0]
                    y1 = tracker.tracks[i].trace[j][1][0]
                    x2 = tracker.tracks[i].trace[j + 1][0][0]
                    y2 = tracker.tracks[i].trace[j + 1][1][0]
                    clr = tracker.tracks[i].track_id % 9
                    distance = ((x2 - x1)** 2 + (y2 - y1)**2)**0.5
                    if distance <  max_point_distance:
                        cv2.line(result, (int(x1), int(y1)), (int(x2), int(y2)),
                                 track_colors[clr], 4)
    return tracker,result


In [6]:
# 加载keras yolov3 voc预训练模型
yolo_test_args = {
    "model_path": 'model_data/yolo.h5',
    "anchors_path": 'model_data/yolo_anchors.txt',
    "classes_path": 'model_data/coco_classes.txt',
    "score" : 0.3,
    "iou" : 0.45,
    "model_image_size" : (416, 416),
    # the model size must be 32*n, else cannot load the project
    "gpu_num" : 1,
}


yolo_test = YOLO(**yolo_test_args)
print("down!")

model_data/yolo.h5 model, anchors, and classes loaded.
down!


In [7]:
# tracker = Tracker(100, 8, 15, 100)
# #for n in tqdm(range(100)):
# image = Image.open('11.jpg')
# r_image,out_boxes, out_scores, out_classes = yolo_test.detect_image(image)
# centers,number = calc_center(out_boxes,out_classes,out_scores,score_limit = 0.5)
# tracker,result = trackerDetection(tracker,r_image,centers,number,max_point_distance = 30,max_colors = 20,track_id_size = 2)
# #misc.imsave('jpg2video/%s.jpg'%n, result)
# plt.imshow(result)

'\n    解析方式一： 从视频保存成的图像文件中进行解析\n    先把视频-> 拆分成图像文件夹，在文件夹中逐帧解析\n'

In [8]:
'''  
    直接读取视频流，并保存在某一个文件夹之中
'''
# 视频 -> 图像
tracker = Tracker(100, 8, 15, 0)
path = "test.mp4"
cap = cv2.VideoCapture(path)#cap仅仅是摄像头的一个对象
fps = cap.get(cv2.CAP_PROP_FPS) #获取视频的帧率
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))#获取视频的大小
# print(size)
fourcc = cv2.VideoWriter_fourcc(*'XVID')#指定编码格式，Windows使用XVID
out = cv2.VideoWriter('Output.avi',fourcc,fps,size,True)
#定义一个视频存储对象，以及视频编码方式,帧率，视频大小格式,最后一项设定灰度图（默认为True彩色，但试了一下改成False视频生成会出错）
# n = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    '''
    ret means return, its output is true or false
    frame means one frame of the video
    '''
#     if frame is None:
#         print("all frames are solved")
#         break
#     if ret is False:
#         print("there are no video sent to the classfier, please check the enter")
#         break
    if ret is True:
        image = Image.fromarray(frame) 
        r_image,out_boxes, out_scores, out_classes = yolo_test.detect_image(image)
        centers,number = calc_center(out_boxes,out_classes,out_scores,score_limit = 0.5)
        tracker,result = trackerDetection(tracker,r_image,centers,number,max_point_distance = 20)
#         print(type(result))
    #misc.imsave('%s.jpg'%n, result)
#     cv2.imwrite('%s.jpg'%n,result, [int(cv2.IMWRITE_JPEG_QUALITY), 100] )
#     fourcc = cv2.VideoWriter_fourcc(*'XVID')  # 保存视频的编码
#         out = cv2.VideoWriter('output.avi',fourcc, fps, size,True)#save the vedio as the teacher given.
        out.write(result)
#         cv2.imshow('the result', result)
#         cv2.waitKey(40)
    else:
        break
#     n += 1
#     print(n)
print('Down!')
cap.release()
out.release()
cv2.destroyAllWindows()


(416, 416, 3)
Found 10 boxes for img
truck 0.51 (1, 150) (40, 215)
truck 0.92 (321, 123) (405, 241)
car 0.37 (2, 144) (39, 211)
car 0.48 (282, 0) (292, 7)
car 0.60 (271, 0) (283, 7)
car 0.62 (290, 13) (306, 28)
car 0.94 (240, 19) (257, 32)
car 0.99 (257, 32) (281, 49)
car 0.99 (219, 53) (251, 82)
person 0.94 (542, 238) (571, 283)
3.79868979999992
(416, 416, 3)
Found 11 boxes for img
truck 0.41 (1, 149) (36, 222)
truck 0.82 (316, 110) (400, 234)
bus 0.31 (316, 110) (400, 234)
car 0.49 (282, 0) (292, 7)
car 0.53 (1, 157) (32, 212)
car 0.58 (271, 0) (283, 7)
car 0.60 (291, 11) (305, 25)
car 0.94 (240, 19) (257, 32)
car 0.99 (257, 32) (281, 48)
car 0.99 (219, 52) (251, 81)
person 0.87 (537, 236) (566, 281)
0.8609742999997252
(416, 416, 3)
Found 10 boxes for img
truck 0.65 (316, 107) (399, 226)
bus 0.65 (314, 111) (398, 219)
car 0.41 (292, 11) (305, 24)
car 0.48 (0, 157) (30, 217)
car 0.49 (282, 0) (292, 7)
car 0.57 (271, 0) (283, 7)
car 0.92 (240, 18) (257, 32)
car 0.99 (258, 32) (281, 48)

Found 9 boxes for img
truck 0.67 (312, 64) (354, 129)
bus 0.68 (312, 64) (354, 129)
car 0.33 (282, 0) (293, 8)
car 0.52 (271, 0) (283, 8)
car 0.54 (263, 21) (280, 38)
car 0.65 (245, 11) (258, 25)
car 0.96 (227, 38) (255, 59)
car 0.96 (215, 182) (282, 250)
person 0.94 (477, 158) (499, 193)
0.8439493000000766
(416, 416, 3)
Found 12 boxes for img
truck 0.64 (312, 60) (353, 124)
bus 0.70 (312, 60) (353, 124)
motorbike 0.64 (480, 179) (497, 195)
car 0.36 (282, 0) (293, 8)
car 0.38 (288, 10) (299, 20)
car 0.59 (271, 0) (283, 8)
car 0.61 (263, 20) (280, 37)
car 0.72 (245, 11) (258, 25)
car 0.83 (218, 166) (286, 240)
car 0.97 (228, 37) (255, 59)
person 0.49 (480, 156) (494, 174)
person 0.87 (476, 156) (497, 192)
0.8674123000000691
(416, 416, 3)
Found 11 boxes for img
truck 0.65 (311, 59) (351, 121)
bus 0.69 (311, 59) (351, 121)
motorbike 0.44 (479, 177) (493, 193)
car 0.37 (283, 0) (292, 8)
car 0.55 (288, 10) (299, 20)
car 0.62 (271, 0) (283, 8)
car 0.68 (263, 20) (280, 37)
car 0.78 (246, 11) 

Found 11 boxes for img
truck 0.72 (305, 40) (337, 84)
bus 0.46 (303, 38) (336, 86)
car 0.42 (249, 10) (259, 21)
car 0.43 (272, 0) (283, 7)
car 0.49 (287, 7) (297, 16)
car 0.60 (283, 0) (293, 8)
car 0.90 (320, 314) (437, 358)
car 0.92 (261, 17) (282, 31)
car 0.97 (235, 29) (255, 46)
car 0.99 (246, 97) (286, 137)
person 0.86 (437, 114) (454, 140)
0.8374331999998503
(416, 416, 3)
Found 11 boxes for img
truck 0.64 (305, 39) (337, 83)
bus 0.61 (303, 37) (336, 84)
car 0.37 (249, 10) (259, 21)
car 0.44 (287, 7) (297, 16)
car 0.45 (272, 0) (283, 7)
car 0.58 (283, 1) (293, 8)
car 0.81 (320, 303) (437, 360)
car 0.91 (261, 16) (282, 31)
car 0.96 (235, 28) (255, 46)
car 0.99 (246, 96) (287, 135)
person 0.74 (436, 114) (454, 139)
0.8656213999997817
(416, 416, 3)
Found 11 boxes for img
truck 0.63 (306, 39) (336, 83)
bus 0.61 (303, 37) (336, 83)
car 0.32 (249, 10) (259, 21)
car 0.43 (287, 7) (297, 16)
car 0.48 (272, 0) (283, 7)
car 0.52 (316, 289) (436, 359)
car 0.57 (283, 1) (293, 8)
car 0.89 (261, 

Found 13 boxes for img
truck 0.32 (314, 142) (372, 208)
truck 0.49 (299, 27) (328, 60)
bus 0.34 (300, 29) (328, 61)
car 0.30 (299, 27) (328, 60)
car 0.31 (186, 0) (197, 7)
car 0.32 (250, 9) (259, 19)
car 0.39 (272, 0) (282, 7)
car 0.70 (283, 1) (294, 9)
car 0.73 (317, 143) (375, 207)
car 0.83 (239, 20) (256, 36)
car 0.88 (266, 10) (282, 25)
car 0.99 (257, 64) (286, 89)
person 0.87 (409, 89) (424, 113)
0.8643607000003612
(416, 416, 3)
Found 11 boxes for img
truck 0.41 (299, 26) (328, 60)
car 0.30 (186, 0) (197, 7)
car 0.32 (250, 9) (259, 19)
car 0.38 (272, 0) (282, 7)
car 0.50 (299, 26) (328, 60)
car 0.74 (283, 1) (293, 9)
car 0.83 (239, 19) (256, 35)
car 0.84 (317, 140) (374, 204)
car 0.89 (265, 10) (282, 25)
car 0.99 (257, 62) (286, 89)
person 0.89 (408, 88) (423, 111)
0.8445482000001903
(416, 416, 3)
Found 10 boxes for img
truck 0.38 (299, 26) (327, 59)
car 0.35 (249, 9) (259, 18)
car 0.37 (272, 0) (282, 7)
car 0.39 (299, 26) (327, 59)
car 0.76 (283, 2) (293, 9)
car 0.77 (315, 143) (

Found 10 boxes for img
bus 0.56 (216, 163) (281, 236)
car 0.50 (212, 167) (279, 234)
car 0.55 (283, 2) (294, 9)
car 0.56 (294, 12) (320, 47)
car 0.70 (248, 9) (260, 21)
car 0.75 (243, 15) (257, 30)
car 0.93 (259, 45) (285, 67)
car 0.95 (304, 89) (346, 127)
car 0.97 (266, 9) (280, 22)
person 0.80 (392, 70) (402, 88)
0.881730300000072
(416, 416, 3)
Found 10 boxes for img
truck 0.33 (294, 12) (319, 46)
car 0.43 (284, 2) (294, 9)
car 0.50 (294, 12) (319, 46)
car 0.67 (243, 15) (258, 29)
car 0.70 (248, 9) (260, 22)
car 0.75 (212, 163) (278, 232)
car 0.93 (306, 82) (345, 124)
car 0.95 (259, 45) (285, 66)
car 0.96 (266, 8) (280, 22)
person 0.80 (390, 69) (402, 88)
0.8444753999997374
(416, 416, 3)
Found 11 boxes for img
truck 0.41 (221, 156) (281, 226)
bus 0.37 (221, 156) (281, 226)
car 0.46 (284, 1) (294, 9)
car 0.50 (217, 158) (282, 225)
car 0.53 (295, 12) (319, 45)
car 0.61 (243, 15) (258, 29)
car 0.73 (248, 9) (260, 23)
car 0.96 (306, 82) (345, 122)
car 0.96 (266, 8) (280, 22)
car 0.97 (26

...

Found 19 boxes for img
bus 0.96 (369, 42) (429, 117)
bus 0.99 (0, 41) (85, 112)
car 0.44 (292, 9) (307, 21)
car 0.47 (157, 4) (183, 25)
car 0.53 (186, 0) (203, 10)
car 0.68 (270, 11) (283, 21)
car 0.73 (283, 8) (300, 21)
car 0.90 (269, 18) (285, 31)
car 0.90 (207, 3) (223, 18)
car 0.91 (327, 23) (358, 49)
car 0.91 (247, 10) (261, 23)
car 0.92 (89, 63) (127, 105)
car 0.97 (205, 75) (240, 109)
car 0.98 (256, 32) (283, 52)
car 0.98 (298, 37) (323, 59)
car 0.99 (305, 66) (339, 92)
car 0.99 (162, 32) (189, 54)
person 0.44 (437, 55) (444, 73)
person 0.58 (466, 103) (479, 126)
0.8780819999992673
(416, 416, 3)
Found 21 boxes for img
bus 0.96 (368, 43) (428, 117)
bus 0.99 (0, 42) (85, 112)
car 0.31 (313, 4) (341, 22)
car 0.32 (208, 1) (221, 12)
car 0.42 (291, 9) (307, 20)
car 0.51 (186, 0) (202, 10)
car 0.59 (156, 4) (183, 25)
car 0.68 (270, 11) (283, 21)
car 0.74 (283, 8) (300, 21)
car 0.88 (326, 23) (357, 49)
car 0.89 (269, 18) (285, 31)
car 0.90 (247, 10) (261, 22)
car 0.92 (207, 3) (222, 18

Found 18 boxes for img
bus 0.93 (365, 39) (416, 108)
bus 0.99 (0, 46) (76, 131)
car 0.40 (291, 9) (307, 20)
car 0.52 (150, 4) (181, 30)
car 0.67 (269, 11) (283, 22)
car 0.75 (201, 9) (215, 22)
car 0.75 (283, 8) (300, 21)
car 0.88 (322, 27) (354, 46)
car 0.90 (44, 85) (110, 137)
car 0.91 (247, 11) (261, 22)
car 0.91 (268, 18) (284, 31)
car 0.95 (304, 58) (334, 81)
car 0.95 (296, 31) (321, 52)
car 0.96 (159, 41) (184, 64)
car 0.97 (257, 31) (283, 48)
car 0.98 (212, 62) (244, 92)
person 0.49 (433, 56) (441, 73)
person 0.82 (453, 98) (465, 122)
0.8733455999999933
(416, 416, 3)
Found 19 boxes for img
truck 0.33 (150, 4) (180, 31)
bus 0.90 (364, 38) (415, 107)
bus 0.99 (0, 47) (75, 130)
car 0.38 (150, 4) (180, 31)
car 0.45 (291, 9) (307, 21)
car 0.67 (269, 11) (283, 22)
car 0.75 (283, 8) (300, 21)
car 0.84 (200, 9) (215, 22)
car 0.88 (322, 26) (354, 46)
car 0.91 (248, 11) (261, 22)
car 0.92 (268, 18) (284, 31)
car 0.94 (296, 31) (321, 52)
car 0.95 (213, 61) (243, 91)
car 0.95 (304, 56) (333,

Found 18 boxes for img
truck 0.46 (143, 4) (175, 33)
truck 0.69 (0, 116) (56, 190)
bus 0.93 (355, 37) (412, 93)
bus 0.96 (0, 50) (55, 126)
car 0.44 (224, 0) (234, 8)
car 0.55 (321, 22) (349, 45)
car 0.71 (268, 10) (282, 22)
car 0.75 (143, 9) (173, 36)
car 0.77 (282, 7) (302, 21)
car 0.92 (248, 10) (261, 23)
car 0.92 (268, 19) (283, 31)
car 0.92 (196, 10) (214, 26)
car 0.96 (258, 29) (282, 46)
car 0.96 (221, 51) (245, 78)
car 0.98 (296, 28) (315, 46)
car 0.98 (149, 49) (180, 76)
car 0.98 (300, 51) (328, 70)
person 0.40 (432, 54) (440, 72)
0.8662920999995549
(416, 416, 3)
Found 21 boxes for img
truck 0.46 (143, 3) (175, 34)
truck 0.66 (0, 123) (55, 199)
bus 0.93 (355, 35) (411, 93)
bus 0.96 (0, 49) (55, 127)
car 0.30 (310, 5) (336, 21)
car 0.42 (224, 0) (233, 7)
car 0.43 (290, 8) (307, 21)
car 0.48 (0, 123) (55, 199)
car 0.58 (321, 22) (348, 45)
car 0.71 (268, 10) (282, 22)
car 0.73 (142, 8) (174, 36)
car 0.77 (282, 7) (301, 21)
car 0.91 (195, 10) (214, 26)
car 0.91 (248, 10) (261, 23)
c

Found 19 boxes for img
bus 0.38 (133, 8) (169, 42)
bus 0.82 (0, 52) (43, 135)
bus 0.96 (350, 32) (405, 85)
car 0.31 (205, 1) (219, 10)
car 0.35 (168, 3) (190, 18)
car 0.44 (222, 0) (232, 8)
car 0.45 (290, 8) (307, 22)
car 0.56 (187, 16) (204, 30)
car 0.70 (322, 17) (346, 39)
car 0.70 (283, 7) (302, 21)
car 0.72 (268, 11) (282, 22)
car 0.90 (268, 19) (283, 31)
car 0.91 (222, 44) (248, 67)
car 0.92 (259, 27) (281, 44)
car 0.93 (248, 10) (261, 23)
car 0.96 (295, 26) (313, 43)
car 0.98 (137, 62) (170, 96)
car 0.98 (297, 45) (327, 65)
person 0.43 (434, 82) (444, 101)
0.8737750999998752
(416, 416, 3)
Found 19 boxes for img
bus 0.55 (133, 8) (168, 42)
bus 0.85 (0, 52) (42, 135)
bus 0.96 (350, 32) (405, 85)
car 0.31 (167, 3) (189, 18)
car 0.35 (205, 1) (219, 10)
car 0.45 (290, 8) (307, 22)
car 0.46 (222, 0) (232, 8)
car 0.67 (322, 17) (346, 39)
car 0.70 (283, 7) (301, 21)
car 0.71 (187, 15) (204, 31)
car 0.73 (268, 11) (282, 22)
car 0.90 (268, 19) (283, 31)
car 0.91 (259, 27) (281, 44)
car 0.9

Found 21 boxes for img
traffic light 0.36 (425, 77) (435, 98)
bus 0.58 (125, 9) (163, 49)
bus 0.60 (0, 57) (36, 135)
bus 0.96 (347, 31) (394, 77)
car 0.33 (204, 0) (220, 10)
car 0.34 (305, 4) (323, 20)
car 0.36 (290, 7) (307, 22)
car 0.37 (220, 1) (233, 10)
car 0.73 (283, 7) (301, 21)
car 0.74 (268, 11) (281, 22)
car 0.81 (317, 16) (345, 38)
car 0.89 (294, 22) (311, 36)
car 0.90 (262, 26) (282, 44)
car 0.90 (267, 19) (283, 32)
car 0.91 (247, 10) (261, 23)
car 0.96 (116, 77) (157, 120)
car 0.98 (223, 40) (250, 59)
car 0.98 (297, 41) (324, 58)
car 0.98 (179, 16) (200, 35)
person 0.31 (425, 77) (435, 98)
person 0.33 (52, 55) (59, 74)
0.8882581000007121
(416, 416, 3)
Found 20 boxes for img
traffic light 0.50 (424, 76) (434, 97)
bus 0.43 (124, 9) (164, 49)
bus 0.71 (0, 57) (35, 135)
bus 0.93 (347, 31) (395, 77)
car 0.30 (305, 4) (323, 20)
car 0.33 (290, 7) (307, 22)
car 0.40 (220, 1) (232, 10)
car 0.43 (200, 4) (214, 15)
car 0.72 (268, 11) (281, 22)
car 0.74 (282, 7) (301, 21)
car 0.82 (317

In [15]:
result.shape

(360, 640, 3)